In [1]:
import simpy
from numpy.random import choice

In [2]:
n=5 #Prime router number
N=3 #regular router number
m=3 #shortest path number

In [3]:
class Packet(object):
    """ A very simple class that represents a packet.
        This packet will run through a queue at a switch output port.
        We use a float to represent the size of the packet in bytes so that
        we can compare to ideal M/M/1 queues.

        Parameters
        ----------
        time : float
            the time the packet arrives at the output queue.
        size : float
            the size of the packet in bytes
        id : int
            an identifier for the packet
        src, dst : int
            identifiers for source and destination

    """
    def __init__(self, packet_id, path, cur_path_ind, time, size, src, dst):
        
        self.packet_id=packet_id
        self.path=path
        self.cur_path_ind=cur_path_ind
        
        
        self.time = time
        self.size = size
        self.src = src
        self.dst = dst


    def __repr__(self):
        return "id: {}, src: {}, time: {}, size: {}".\
            format(self.packet_id, self.src, self.time, self.size)

'''path, cur_path_ind, '''

In [4]:
class PacketGenerator(object):
    """ Generates packets with given inter-arrival time distribution.
        Set the "out" member variable to the entity to receive the packet.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        adist : function
            a no parameter function that returns the successive inter-arrival times of the packets
        sdist : function
            a no parameter function that returns the successive sizes of the packets
        initial_delay : number
            Starts generation after an initial delay. Default = 0
        finish : number
            Stops generation at the finish time. Default is infinite


    """
    def __init__(self, env, gen_prob, generator_id,  adist, sdist, initial_delay=0, finish=float("inf")):
        
        self.gen_prob=gen_prob
        self.generator_id = generator_id
        self.env = env
        self.adist = adist
        self.sdist = sdist
        self.initial_delay = initial_delay
        self.finish = finish
        self.out = None
        self.packets_sent = 0
        self.action = env.process(self.run())  # starts the run() method as a SimPy process


    def run(self):
        """The generator function used in simulations.
        """
        
        yield self.env.timeout(self.initial_delay)
        while self.env.now < self.finish:
            # wait for next transmission
            yield self.env.timeout(self.adist())
            
            destination = choice(range(1,N+1), p=self.gen_prob)
            #print('destination',destination)
            #print('id',self.generator_id)
            opt_path=path[self.generator_id][destination]
            
           #print('opt_path',opt_path)
            
            self.packets_sent += 1
            
            
            pkt = Packet(packet_id=self.packets_sent, 
                         path=opt_path, 
                         cur_path_ind=0, 
                         time=self.env.now, 
                         size=self.sdist(), 
                         src=self.generator_id, 
                         dst=destination)
            
            #if next_node==destination:
            #    sink[destination].put(p)
            #else:
            #    switch[destination].put(p)
            
            self.out.put(pkt)
       

In [5]:
class PacketSink(object):
    """ Receives packets and collects delay information into the
        waits list. You can then use this list to look at delay statistics.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        debug : boolean
            if true then the contents of each packet will be printed as it is received.
        rec_arrivals : boolean
            if true then arrivals will be recorded
        absolute_arrivals : boolean
            if true absolute arrival times will be recorded, otherwise the time between consecutive arrivals
            is recorded.
        rec_waits : boolean
            if true waiting time experienced by each packet is recorded
        selector: a function that takes a packet and returns a boolean
            used for selective statistics. Default none.

    """
    def __init__(self, env, rec_arrivals=False, absolute_arrivals=False, rec_waits=True, debug=False, selector=None):
        self.store = simpy.Store(env)
        self.env = env
        self.rec_waits = rec_waits
        self.rec_arrivals = rec_arrivals
        self.absolute_arrivals = absolute_arrivals
        self.waits = []
        self.arrivals = []
        self.debug = debug
        self.packets_rec = 0
        self.bytes_rec = 0
        self.selector = selector
        self.last_arrival = 0.0

    def put(self, pkt):
        #print('packet_id',pkt.id)
        print('sink haha', pkt.packet_id, pkt.src)
        
        if not self.selector or self.selector(pkt):
            now = self.env.now
            if self.rec_waits:
                self.waits.append(self.env.now - pkt.time)
            if self.rec_arrivals:
                if self.absolute_arrivals:
                    self.arrivals.append(now)
                else:
                    self.arrivals.append(now - self.last_arrival)
                self.last_arrival = now
            self.packets_rec += 1
            self.bytes_rec += pkt.size
            if self.debug:
                print(pkt)

In [6]:
def put_in_queue(i,pkt):
    
    
    switch[i].packets_rec += 1
    tmp_byte_count = switch[i].byte_size + pkt.size

    #if switch[i].qlimit is None:
    #    switch[i].byte_size = tmp_byte_count
    #    return switch[i].store.put(pkt)
    if switch[i].limit_bytes and tmp_byte_count >= switch[i].qlimit:
        switch[i].packets_drop += 1
        return
    #elif not switch[i].limit_bytes and len(switch[i].store.items) >= switch[i].qlimit-1:
    #    switch[i].packets_drop += 1
    else:
        switch[i].byte_size = tmp_byte_count
        switch[i].store.put(pkt)
        #print('router', i, 'store', self.store.items)
        return 

In [7]:
class SwitchPort(object):
    """ Models a switch output port with a given rate and buffer size limit in bytes.
        Set the "out" member variable to the entity to receive the packet.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        rate : float
            the bit rate of the port
        qlimit : integer (or None)
            a buffer size limit in bytes or packets for the queue (including items
            in service).
        limit_bytes : If true, the queue limit will be based on bytes if false the
            queue limit will be based on packets.

    """
    def __init__(self, env, router_id, rate, qlimit=None, limit_bytes=True, debug=False):
        self.router_id=router_id
        self.store = simpy.Store(env)
        self.rate = rate
        self.env = env
        self.out = None
        self.packets_rec = 0
        self.packets_drop = 0
        self.qlimit = qlimit
        self.limit_bytes = limit_bytes
        self.byte_size = 0  # Current size of the queue in bytes
        self.debug = debug
        self.busy = 0  # Used to track if a packet is currently being sent
        self.action = env.process(self.run())  # starts the run() method as a SimPy process

    def run(self):

        while True:
            msg = (yield self.store.get())
            self.busy = 1
            self.byte_size -= msg.size
            yield self.env.timeout(msg.size*8.0/self.rate)
            
            #print('msg', msg.packet_id, msg.src, 'router', self.router_id)
            #self.out=switch[1]
            #self.out.put(msg)
            #self.busy = 0
            #if self.debug:
            #    print(msg)
 
            path=msg.path
            cur_path_ind=msg.cur_path_ind
            destination=msg.dst
                
            #print('path',path)
            #print('cur_path_ind',cur_path_ind)
            #print('destination',destination)
            #print('path[cur_path_ind]',path[cur_path_ind])
            
            #self.packet_id=packet_id
            #self.path=path
            #self.cur_path_ind=cur_path_ind
            #print('msg packet_id src ',msg.packet_id, msg.src)
                
            if path[cur_path_ind]==destination:
                #print('send to sink' + str(destination))
                sink.put(msg)
            else:
                next_node=path[cur_path_ind+1]
                #print('send to switch' + str(next_node))
                msg.cur_path_ind=cur_path_ind+1
                #print('msg.cur_path_ind_after',msg.cur_path_ind)
                
                #put_in_queue(next_node,msg)
                
                
                switch[next_node].store.put(msg)   
                
                #print('msg', msg.packet_id, msg.src)
                #switch[next_node].put(msg)
                
                #switch_for_switches.put((msg, next_node))
                
            self.busy = 0
            #if self.debug:
            #    print(msg)
                
     

    def put(self, pkt):
        #put_in_queue(i,pkt)
        self.packets_rec += 1
        tmp_byte_count = self.byte_size + pkt.size

        #if switch[i].qlimit is None:
        #    switch[i].byte_size = tmp_byte_count
        #    return switch[i].store.put(pkt)
        if self.limit_bytes and tmp_byte_count >= self.qlimit:
            self.packets_drop += 1
            return
        #elif not switch[i].limit_bytes and len(switch[i].store.items) >= switch[i].qlimit-1:
        #    switch[i].packets_drop += 1
        else:
            self.byte_size = tmp_byte_count
            self.store.put(pkt)
            #prifnt('router', i, 'store', self.store.items)
            return     
                

In [8]:
from random import expovariate
import simpy

def constArrival():  # Constant arrival distribution for generator 1
    return 1.5

def constArrival2():
    return 2.0

def distSize():
    return expovariate(0.01)

In [9]:
def constArrival():
    return 1.5    # time interval

def constSize():
    return 100.0  # bytes

In [10]:
path=[0]*4
prob=[0]*4
for i in range(4):
    path[i]=[0]*4


path[1][2]=[1,3,2]
path[2][3]=[2,1,3]
path[3][1]=[3,2,1]
path[2][1]=[2,3,1]
path[3][2]=[3,1,2]
path[1][3]=[1,2,3]

gen_prob=[0]*4
gen_prob[1]=[0,0.5,0.5]
gen_prob[2]=[0.2,0,0.8]
gen_prob[3]=[0.7,0.3,0]

In [11]:
path

[[0, 0, 0, 0],
 [0, 0, [1, 3, 2], [1, 2, 3]],
 [0, [2, 3, 1], 0, [2, 1, 3]],
 [0, [3, 2, 1], [3, 1, 2], 0]]

In [12]:
import numpy as np

generators=[0]*4
#switch=np.array([0]*4)
switch=[0]*4

env = simpy.Environment()  # Create the SimPy environment

sink=PacketSink(env, debug=True)



#switchx=SwitchPort(env, 100, rate=200.0, qlimit=300, debug=True)
#switch_for_switches=SwitchForSwitches(env)



for i in range(1,4):
    generators[i]=PacketGenerator(env, gen_prob[i], i, constArrival, constSize)
    switch[i]=SwitchPort(env=env, router_id=i, rate=1000.0, qlimit=30000, debug=True)
    #switch[i].out=
    generators[i].out = switch[i]
    #switch[i].out=switchx
    #sink[i]=PacketSink(env, debug=True)
    

#switchx.out=sink
    
env.run(until=2000)
print("sink waits: {}".format(sink.waits))

for i in range(1,4):
    print("{} received: {}, dropped {}, sent {}".format(i, sink.packets_rec,
         switch[i].packets_drop, generators[i].packets_sent))

sink haha 1 1
id: 1, src: 1, time: 1.5, size: 100.0
sink haha 1 3
id: 1, src: 3, time: 1.5, size: 100.0
sink haha 1 2
id: 1, src: 2, time: 1.5, size: 100.0
sink haha 2 3
id: 2, src: 3, time: 3.0, size: 100.0
sink haha 2 1
id: 2, src: 1, time: 3.0, size: 100.0
sink haha 2 2
id: 2, src: 2, time: 3.0, size: 100.0
sink haha 3 1
id: 3, src: 1, time: 4.5, size: 100.0
sink haha 3 2
id: 3, src: 2, time: 4.5, size: 100.0
sink haha 3 3
id: 3, src: 3, time: 4.5, size: 100.0
sink haha 4 3
id: 4, src: 3, time: 6.0, size: 100.0
sink haha 4 2
id: 4, src: 2, time: 6.0, size: 100.0
sink haha 5 1
id: 5, src: 1, time: 7.5, size: 100.0
sink haha 4 1
id: 4, src: 1, time: 6.0, size: 100.0
sink haha 5 3
id: 5, src: 3, time: 7.5, size: 100.0
sink haha 5 2
id: 5, src: 2, time: 7.5, size: 100.0
sink haha 6 2
id: 6, src: 2, time: 9.0, size: 100.0
sink haha 6 1
id: 6, src: 1, time: 9.0, size: 100.0
sink haha 6 3
id: 6, src: 3, time: 9.0, size: 100.0
sink haha 7 2
id: 7, src: 2, time: 10.5, size: 100.0
sink haha 7

sink haha 116 2
id: 116, src: 2, time: 174.0, size: 100.0
sink haha 120 3
id: 120, src: 3, time: 180.0, size: 100.0
sink haha 118 1
id: 118, src: 1, time: 177.0, size: 100.0
sink haha 119 3
id: 119, src: 3, time: 178.5, size: 100.0
sink haha 120 1
id: 120, src: 1, time: 180.0, size: 100.0
sink haha 117 2
id: 117, src: 2, time: 175.5, size: 100.0
sink haha 119 1
id: 119, src: 1, time: 178.5, size: 100.0
sink haha 121 3
id: 121, src: 3, time: 181.5, size: 100.0
sink haha 118 2
id: 118, src: 2, time: 177.0, size: 100.0
sink haha 122 3
id: 122, src: 3, time: 183.0, size: 100.0
sink haha 121 1
id: 121, src: 1, time: 181.5, size: 100.0
sink haha 123 3
id: 123, src: 3, time: 184.5, size: 100.0
sink haha 119 2
id: 119, src: 2, time: 178.5, size: 100.0
sink haha 122 1
id: 122, src: 1, time: 183.0, size: 100.0
sink haha 124 3
id: 124, src: 3, time: 186.0, size: 100.0
sink haha 123 1
id: 123, src: 1, time: 184.5, size: 100.0
sink haha 120 2
id: 120, src: 2, time: 180.0, size: 100.0
sink haha 125 

sink haha 242 1
id: 242, src: 1, time: 363.0, size: 100.0
sink haha 241 2
id: 241, src: 2, time: 361.5, size: 100.0
sink haha 234 3
id: 234, src: 3, time: 351.0, size: 100.0
sink haha 242 2
id: 242, src: 2, time: 363.0, size: 100.0
sink haha 243 1
id: 243, src: 1, time: 364.5, size: 100.0
sink haha 239 3
id: 239, src: 3, time: 358.5, size: 100.0
sink haha 247 2
id: 247, src: 2, time: 370.5, size: 100.0
sink haha 241 1
id: 241, src: 1, time: 361.5, size: 100.0
sink haha 235 3
id: 235, src: 3, time: 352.5, size: 100.0
sink haha 240 3
id: 240, src: 3, time: 360.0, size: 100.0
sink haha 243 2
id: 243, src: 2, time: 364.5, size: 100.0
sink haha 236 3
id: 236, src: 3, time: 354.0, size: 100.0
sink haha 244 2
id: 244, src: 2, time: 366.0, size: 100.0
sink haha 244 1
id: 244, src: 1, time: 366.0, size: 100.0
sink haha 241 3
id: 241, src: 3, time: 361.5, size: 100.0
sink haha 245 1
id: 245, src: 1, time: 367.5, size: 100.0
sink haha 245 2
id: 245, src: 2, time: 367.5, size: 100.0
sink haha 243 

sink haha 359 2
id: 359, src: 2, time: 538.5, size: 100.0
sink haha 366 1
id: 366, src: 1, time: 549.0, size: 100.0
sink haha 362 1
id: 362, src: 1, time: 543.0, size: 100.0
sink haha 367 1
id: 367, src: 1, time: 550.5, size: 100.0
sink haha 361 3
id: 361, src: 3, time: 541.5, size: 100.0
sink haha 362 2
id: 362, src: 2, time: 543.0, size: 100.0
sink haha 361 2
id: 361, src: 2, time: 541.5, size: 100.0
sink haha 368 1
id: 368, src: 1, time: 552.0, size: 100.0
sink haha 362 3
id: 362, src: 3, time: 543.0, size: 100.0
sink haha 364 1
id: 364, src: 1, time: 546.0, size: 100.0
sink haha 366 3
id: 366, src: 3, time: 549.0, size: 100.0
sink haha 364 2
id: 364, src: 2, time: 546.0, size: 100.0
sink haha 363 3
id: 363, src: 3, time: 544.5, size: 100.0
sink haha 369 1
id: 369, src: 1, time: 553.5, size: 100.0
sink haha 363 2
id: 363, src: 2, time: 544.5, size: 100.0
sink haha 365 2
id: 365, src: 2, time: 547.5, size: 100.0
sink haha 367 3
id: 367, src: 3, time: 550.5, size: 100.0
sink haha 366 

sink haha 484 2
id: 484, src: 2, time: 726.0, size: 100.0
sink haha 485 3
id: 485, src: 3, time: 727.5, size: 100.0
sink haha 481 1
id: 481, src: 1, time: 721.5, size: 100.0
sink haha 486 3
id: 486, src: 3, time: 729.0, size: 100.0
sink haha 482 1
id: 482, src: 1, time: 723.0, size: 100.0
sink haha 484 3
id: 484, src: 3, time: 726.0, size: 100.0
sink haha 486 2
id: 486, src: 2, time: 729.0, size: 100.0
sink haha 487 3
id: 487, src: 3, time: 730.5, size: 100.0
sink haha 490 2
id: 490, src: 2, time: 735.0, size: 100.0
sink haha 483 1
id: 483, src: 1, time: 724.5, size: 100.0
sink haha 487 2
id: 487, src: 2, time: 730.5, size: 100.0
sink haha 487 1
id: 487, src: 1, time: 730.5, size: 100.0
sink haha 488 3
id: 488, src: 3, time: 732.0, size: 100.0
sink haha 488 1
id: 488, src: 1, time: 732.0, size: 100.0
sink haha 484 1
id: 484, src: 1, time: 726.0, size: 100.0
sink haha 488 2
id: 488, src: 2, time: 732.0, size: 100.0
sink haha 490 3
id: 490, src: 3, time: 735.0, size: 100.0
sink haha 485 

sink haha 602 2
id: 602, src: 2, time: 903.0, size: 100.0
sink haha 611 3
id: 611, src: 3, time: 916.5, size: 100.0
sink haha 599 1
id: 599, src: 1, time: 898.5, size: 100.0
sink haha 602 1
id: 602, src: 1, time: 903.0, size: 100.0
sink haha 603 2
id: 603, src: 2, time: 904.5, size: 100.0
sink haha 608 3
id: 608, src: 3, time: 912.0, size: 100.0
sink haha 601 1
id: 601, src: 1, time: 901.5, size: 100.0
sink haha 613 3
id: 613, src: 3, time: 919.5, size: 100.0
sink haha 603 1
id: 603, src: 1, time: 904.5, size: 100.0
sink haha 610 2
id: 610, src: 2, time: 915.0, size: 100.0
sink haha 606 2
id: 606, src: 2, time: 909.0, size: 100.0
sink haha 612 3
id: 612, src: 3, time: 918.0, size: 100.0
sink haha 605 1
id: 605, src: 1, time: 907.5, size: 100.0
sink haha 614 3
id: 614, src: 3, time: 921.0, size: 100.0
sink haha 604 1
id: 604, src: 1, time: 906.0, size: 100.0
sink haha 607 2
id: 607, src: 2, time: 910.5, size: 100.0
sink haha 606 1
id: 606, src: 1, time: 909.0, size: 100.0
sink haha 612 

In [13]:
from numpy.random import choice

In [14]:
prob=[0.5,0.5,0]

In [15]:
draw = choice([3,4,5], 
              p=prob)

In [16]:
draw

4